In [ ]:
import ee
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import plotly

from waveletFunctions import wavelet

In [ ]:
def get_ndvi_series(lon, lat, init='2007-01-01', end='2019-12-31', scale=500):
    # transform lat/lon to a point
    pnt = ee.Geometry.Point(lon, lat)

    # find all images between init an end that have the requested point
    dataset = ee.ImageCollection('MODIS/006/MOD13Q1')
    dataset = dataset.filterDate(init, end).filterBounds(pnt)

    ndvi = dataset.select('NDVI')
    data = ee.List(ndvi.getRegion(geometry=pnt, scale=scale)).getInfo()

    # extract ndvi series to a pandas dataframe 
    df = pd.DataFrame(data[1:], columns=data[0])
    df['datetime'] = pd.to_datetime(df['time'], unit='ms', utc=True)
    df = df.sort_values('datetime')
    df = df[['datetime', 'NDVI']].set_index('datetime')

    df['NDVI'] = df['NDVI'] / 10000.

    df.interpolate(inplace=True)

    return df

def wavelet_figure(df, hfreq=90, lfreq=365):
    x_raw = df['NDVI'].values
    mean = np.mean(x_raw)
    x = x_raw - mean
    variance_raw = np.std(x, ddof=1) ** 2

    variance = 1.0
    x = x / np.std(x, ddof=1)
    n = len(x)
    dt = 16
    time = np.arange(len(x)) * dt / 365. + 2007.  # construct time array
    pad = 1      # pad the time series with zeroes (recommended)
    dj = 0.25    # this will do 4 sub-octaves per octave
    s0 = 2.*dt    # this says start at a scale of 6 months
    j1 = 5./dj    # this says do 7 powers-of-two with dj sub-octaves each
    lag1 = 0.72  # lag-1 autocorrelation for red noise background
    mother = 'MORLET'

    # Wavelet transform:
    wave, period, scale, coi = wavelet(x, dt, pad, dj, s0, j1, mother)
    power = (np.abs(wave)) ** 2  # compute wavelet power spectrum
    global_ws = (np.sum(power, axis=1) / n) # time-average over all times

    ########################
    #  Spectrum
    ########################
    powers=np.zeros_like(power)
    for k in range(len(scale)):
        powers[k,:] = power[k,:]/scale[k]

    ########################
    #  Global Spectrum
    ########################
    global_wss = global_ws / scale 

    # Scale-average between El Nino periods of 2--8 years
    avg = (scale >= hfreq) & (scale <= lfreq) 
    Cdelta = 0.776  # this is for the MORLET wavelet
    psi00 = np.pi ** (-1. / 4.)

    # filterpower
    filter_series = np.dot(scale.reshape(len(scale),1),np.ones((1,n)))
    filter_series = np.real(wave) / np.sqrt(filter_series)
    filter_series = (dj * (dt ** (1./2.)) / (Cdelta * psi00)) * filter_series[avg, :].sum(axis=0)

    #figure size
    fig, ax = plt.subplots(figsize=(14,7))

    divider = make_axes_locatable(ax)
    axSeries = divider.append_axes("top", size=1.2, pad=0.2, sharex=ax)
    axFourier = divider.append_axes("right", size=1.2, pad=0.3, sharey=ax)
    cax = divider.append_axes("bottom", size=0.2, pad=0.5)

    axSeries.plot(time, x_raw, color='k')
    axSeries.plot(time, filter_series * np.sqrt(variance_raw) + mean, 'r-')

    levels = np.linspace(np.percentile(powers, 10), np.percentile(powers, 90), 8)
    cf = ax.contourf(time, period, powers, levels, extend='both', cmap='viridis')
    #ax.plot(time, coi, 'k')

    axFourier.plot(global_wss, period,"r-")

    ax.margins(0)
    ax.set_yscale('log', basey=2, subsy=None)
    ax.set_ylim([np.min(period), np.max(period)])
    ax.invert_yaxis()
    ax.set_ylabel('Period (days)')
    ax.set_xlabel('Time')
    ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
    ax.ticklabel_format(axis='y', style='plain')

    axSeries.tick_params(labelbottom=False)
    axSeries.set_ylabel(r'NDVI')

    axFourier.set_yscale('log', basey=2, subsy=None)
    axFourier.set_xlabel(r'Global wavelet spectrum')
    axFourier.invert_yaxis()
    axFourier.tick_params(labelleft=False)
    axFourier.yaxis.set_major_formatter(ticker.ScalarFormatter())
    axFourier.ticklabel_format(axis='y', style='plain')
    axFourier.set_xlim([0, 1.25 * np.max(global_wss)])

    cbar = plt.colorbar(cf, cax=cax, orientation='horizontal')
    cbar.set_label(r'Wavelet absolute spectrum')
    plt.tight_layout()

In [ ]:
lon, lat = -52.578321, -14.975312
init, end ='2007-01-01', '2019-12-31'

ee.Initialize()


df = get_ndvi_series(lon, lat, init=init, end=end)


data = {
    'data': [
        {'x': df.index,
        'y': df['NDVI']}
    ],
    'layout': {'title': 'NDVI timeseries at {} {}'.format(lon, lat)}
}

plotly.offline.iplot(data)

In [ ]:
wavelet_figure(df)